## Dataset Maker

In [1]:
import re
import numpy as np
import pymorphy3

from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump
import pickle

In [2]:
def save_file(name: str, obj: list, path: str):
    """ Write prepared data to files saving data structure """
    with open('{path}/{name}.pickle'.format(name=name, path=path), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
special_characters = ['</p>', '\xa0', '<br/>', '\n', '-&gt;']

stop_words = ['в', 'на', 'с', 'и', 'я', 'о', 'к', 'а', 'за', 'по',
              'об', 'у', 'бы', 'от', 'ли', 'ул', 'что', 'со', 'из',
              'для', 'про', 'г', 'до', 'то', 'быть', 'по', 'мочь',
              'январь','февраль','март','апрель','май','июнь','июль',
              'август','сентябрь','октябрь','ноябрь','декабрь','год',
              'стать','это','когда','даже', 'янв', 'фев', 'апр', 'авг',
              'сен', 'окт', 'ноя', 'дек', 'ые', 'ый', 'ым', 'ых', 'ая']


### Reading data

In [4]:
# Reading data from files and writing it into variables

f = open("data/reviews_positive.txt", "r")
positive_reviews = f.read().split('<p>')
positive_reviews.pop(0)

f = open("data/reviews_neutral.txt", "r")
neutral_reviews = f.read().split('<p>')
neutral_reviews.pop(0)

f = open("data/reviews_negative.txt", "r")
negative_reviews = f.read().split('<p>')
negative_reviews.pop(0)

''

### Data processing

In [5]:
def cleansing(text: list) -> list:
    """ Removes numbers, special characters
        and Roman alphabet characters from raw data.
        Converts all characters to lowercase """
    res = []
    for i in range(len(text)):
        res.append(text[i])
        for sp_ch in special_characters:
            res[i] = (text[i].replace(sp_ch,' '))
        res[i] = res[i].lower()
        res[i] = re.sub(r'[^\w\s]', ' ', res[i])
        res[i] = re.sub(r'[^\w\s]+|[\d]+', r' ',res[i]).strip()
        res[i] = re.sub('[a-zA-Z\s]+', ' ', res[i])
        res[i] = re.sub(" +", " ", res[i])

    return res

In [6]:
morph = pymorphy3.MorphAnalyzer()

def lemmatize(sentence: str) -> list:
    """ Converts each word in a sentence to its initial form"""
    words = sentence.split()  # Split sentence into words
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

In [7]:
def delete_stop_words(sentence: list) -> list:
    """ Removes stop-words from a sentence """
    res = list()
    for word in sentence:
        if word not in stop_words:
            res.append(word)
    return res

In [8]:
def join(sentence: list) -> str:
    """ Joins list sentence into a string """
    return ' '.join(sentence)

In [9]:
def processing_pipeline(reviews):
    """ Passes raw data through all processing methods """
    cleansed_reviews = cleansing(reviews)
    lemmatized_reviews = list(map(lemmatize,cleansed_reviews))
    clean_reviews = list(map(delete_stop_words,lemmatized_reviews))
    final_reviews = list(map(join,clean_reviews))
    return final_reviews

### Data preparation

In [10]:
positive_data = processing_pipeline(positive_reviews)
neutral_data = processing_pipeline(neutral_reviews)
negative_data = processing_pipeline(negative_reviews)

In [11]:
print('Raw data: \n')
negative_reviews[0]

Raw data: 



'Перешёл по ссылке друга оформил карту получил в ближайшем банке активно пользуюсь.в причине отказа оформления кэшбека пишут-вы не получили карту (просто недоумеваю).звонил на горячую линию ,обещали разобраться в течении 5 рабочих дней-прошло две недели.берут на измор,пока сам не отстанеш.</p>\n'

In [12]:
print('Final data: \n')
negative_data[0]

Final data: 



'перейти ссылка друг оформить карта получить близкий банк активно пользоваться причина отказ оформление кэшбек писать вы не получить карта просто недоумевать звонить горячий линия обещать разобраться течение рабочий день пройти два неделя брать измор пока сам не отстанеш'

### Data vectorization

In [44]:
# Making final corpus
data = np.concatenate((positive_data,neutral_data))
data = np.concatenate((data,negative_data))

# Making TF-IDF matrix from the resulting corpus
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

# Saving vectorizer
dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

### Save final data

In [45]:
save_file('data_positive', positive_data, 'prepared_data')
save_file('data_neutral', neutral_data, 'prepared_data')
save_file('data_negative', negative_data, 'prepared_data')